### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = des_cipher('0')
samples_count = 100000
text_length = 16

set_characters(letters)
model_path = f'models/{encrypt.name}_{len(letters)}x{text_length}_best_model.h5'

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text, False)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, activation='relu', return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Bidirectional(GRU(64, activation='relu')))
model.add(Dense(text_length, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 24, 256)           99840     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               123264    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
Total params: 225,168
Trainable params: 225,168
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=100, callbacks=callbacks)

Train on 90000 samples, validate on 10000 samples
Epoch 1/100
90000/90000 [==============================] - 334s 4ms/step - loss: 323.6904 - acc: 0.0630 - val_loss: 324.0889 - val_acc: 0.0585

Epoch 00001: val_acc improved from -inf to 0.05850, saving model to models/des_cipher_62x16_best_model.h5
Epoch 2/100
90000/90000 [==============================] - 364s 4ms/step - loss: 321.5770 - acc: 0.0625 - val_loss: 321.5467 - val_acc: 0.0568

Epoch 00002: val_acc did not improve from 0.05850
Epoch 3/100
90000/90000 [==============================] - 348s 4ms/step - loss: 321.3194 - acc: 0.0614 - val_loss: 321.8095 - val_acc: 0.0634

Epoch 00003: val_acc improved from 0.05850 to 0.06340, saving model to models/des_cipher_62x16_best_model.h5
Epoch 4/100
90000/90000 [==============================] - 343s 4ms/step - loss: 321.1393 - acc: 0.0622 - val_loss: 322.8319 - val_acc: 0.0606

Epoch 00004: val_acc did not improve from 0.06340
Epoch 5/100
90000/90000 [==============================] - 

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HelloWorldhello1')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print(f'\nTest cipher vs encrypted prediction:\n{test_cyhper}\n{encrypt(prediction)}')
match_percentage(test_cyhper, encrypt(prediction))

Model predicted that original text is: vvwwuvvvvuvvwwvw

Test cipher vs encrypted prediction:
nr-Hdcor9P28UjS1q8AfjQ==
OFvEweE_yZ0h0WGjmie_Lw==


8.333333333333332